# Deep learning models for age prediction on EEG data

This notebook uses deep learning methods to predict the age of infants using EEG data. The EEG data is preprocessed as shown in the notebook 'Deep learning EEG_dataset preprocessing raw'. 

In [1]:
import sys, os, fnmatch, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.dirname(os.getcwd()))

from config import PATH_DATA_PROCESSED_DL_REDUCED, PATH_MODELS

## Load preprocessed data (reduced)

The data can be found in 'PATH_DATA_PROCESSED_DL_REDUCED'. This is a single folder with all the data and metadata. The EEG data is in the .zarr files and the metadata is in .csv files. The .zarr files are divided in chunks of 1-second epochs (average of 10 original EEG epochs) from the same subject and the metadata contains the information like the subject's identification code and age. The notebook "Deep learning EEG_DL dataset_reduction.ipynb" takes care of reducing the original processed data set to the reduced size.

Generator loads all the data into memory. The generators generate averaged epochs on the fly. The data is split in train/validation/test and no subject can be found in more than one of these splits. 

Originally, we used the original EEG epochs and averaged 30 of them into a new EEG epoch in the generator object. This had two disadvantages: (1) many files had to be openened and closed during the training process, and (2) the data set was too large to fit into memory. Therefore, we decided to randomly create chunks of 10 EEG epochs and average those for each subject/age group. This reduced the data set from ±145GB to ±14.5 GB. We now use these already averaged EEG epochs as "original" epcohs, and average those another 3-5 times to reduce noise. We have also experimented with averaging all EEG epochs of a subject at a specific age into a single EEG epoch, but performance was lower, most likely because this reduced the size of the data set a lot.

In [2]:
%%time

# Load all the metadata

from sklearn.model_selection import train_test_split

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_DL_REDUCED)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(os.path.join(PATH_DATA_PROCESSED_DL_REDUCED, file_name))[0] for file_name in fnmatch.filter(file_names, "*.zarr")]

# Step 3: Load all the metadata
frames = []

for idx, feature_file in enumerate(files):
    df_metadata = pd.read_csv(feature_file + ".csv")
    frames.append(df_metadata)

df_metadata = pd.concat(frames) 

# Step 4: Add missing age information based on the age group the subject is in
df_metadata['age_months'].fillna(df_metadata['age_group'], inplace=True)
df_metadata['age_days'].fillna(df_metadata['age_group']*30, inplace=True)
df_metadata['age_years'].fillna(df_metadata['age_group']/12, inplace=True)

# Step 5: List all the unique subject IDs
subject_ids = sorted(list(set(df_metadata["code"].tolist())))

CPU times: user 2.98 s, sys: 478 ms, total: 3.46 s
Wall time: 4.18 s


In [3]:
from sklearn.model_selection import train_test_split

IDs_train, IDs_temp = train_test_split(subject_ids, test_size=0.3, random_state=42)
IDs_test, IDs_val = train_test_split(IDs_temp, test_size=0.5, random_state=42)

In [4]:
df_metadata

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,146,/Volumes/Seagate Expansion Drive/ePodium/Data/...,146_29,29,860.0,28.666667,2.388889
0,156,/Volumes/Seagate Expansion Drive/ePodium/Data/...,156_29,29,870.0,29.000000,2.416667
0,713,/Volumes/Seagate Expansion Drive/ePodium/Data/...,713_29,29,868.0,28.933333,2.411111
0,411,/Volumes/Seagate Expansion Drive/ePodium/Data/...,411_41,41,1231.0,41.033333,3.419444
0,703,/Volumes/Seagate Expansion Drive/ePodium/Data/...,703_29,29,866.0,28.866667,2.405556
...,...,...,...,...,...,...,...
0,145,/Volumes/Seagate Expansion Drive/ePodium/Data/...,145_41,41,1232.0,41.066667,3.422222
0,124,/Volumes/Seagate Expansion Drive/ePodium/Data/...,124_47,47,1407.0,46.900000,3.908333
0,710,/Volumes/Seagate Expansion Drive/ePodium/Data/...,710_41,41,1234.0,41.133333,3.427778
0,182,/Volumes/Seagate Expansion Drive/ePodium/Data/...,182_23,23,697.0,23.233333,1.936111


In [5]:
from dataset_generator_reduced import DataGeneratorReduced

In [41]:
# # Import libraries
# from tensorflow.keras.utils import Sequence
# import numpy as np
# import zarr
# import os

# class DataGeneratorReduced(Sequence):
#     """Generates data for loading (preprocessed) EEG timeseries data.
#     Create batches for training or prediction from given folders and filenames.

#     """
#     def __init__(self,
#                  list_IDs,
#                  BASE_PATH,
#                  metadata,
#                  gaussian_noise=0.0,
#                  n_average = 3,
#                  batch_size=10,
#                  iter_per_epoch = 1,
#                  n_timepoints = 501,
#                  n_channels=30,
#                  shuffle=True,
#                  warnings=False):
#         """Initialization

#         Args:
#         --------
#         list_IDs:
#             list of all filename/label ids to use in the generator
#         metadata:
#             DataFrame containing all the metadata.
#         n_average: int
#             Number of EEG/time series epochs to average.
#         batch_size:
#             batch size at each iteration
#         iter_per_epoch: int
#             Number of iterations over all data points within one epoch.
#         n_timepoints: int
#             Timepoint dimension of data.
#         n_channels:
#             number of input channels
#         shuffle:
#             True to shuffle label indexes after every epoch
#         """
#         self.list_IDs = list_IDs
#         self.BASE_PATH = BASE_PATH
#         self.metadata = metadata
#         self.metadata_temp = None
#         self.gaussian_noise = gaussian_noise
#         self.n_average = n_average
#         self.batch_size = batch_size
#         self.iter_per_epoch = iter_per_epoch
#         self.n_timepoints = n_timepoints
#         self.n_channels = n_channels
#         self.shuffle = shuffle
#         self.warnings = warnings
#         self.on_epoch_end()

#         # Store all data in here
#         self.X_data_all = []
#         self.y_data_all = []
#         self.load_all_data()


#     def __len__(self):
#         """Denotes the number of batches per epoch

#         return: number of batches per epoch
#         """
#         return int(np.floor(len(self.metadata_temp) / self.batch_size))


#     def __getitem__(self, index):
#         """Generate one batch of data

#         Args:
#         --------
#         index: int
#             index of the batch

#         return: X and y when fitting. X only when predicting
#         """
        
#         # Generate indexes of the batch
#         indexes = self.indexes[index * self.batch_size:((index + 1) * self.batch_size)]

#         # Generate data
#         X, y = self.generate_data(indexes)

#         return X, y

#     def load_all_data(self):
#         """ Loads all data into memory. """
#         for i, metadata_file in self.metadata_temp.iterrows():
#             filename = os.path.join(self.BASE_PATH, metadata_file['cnt_file'] + '.zarr')
            
#             X_data = np.zeros((0, self.n_channels, self.n_timepoints))

#             data_signal = self.load_signal(filename)

#             if (len(data_signal) == 0) and self.warnings:
#                 print(f"EMPTY SIGNAL, filename: {filename}")

#             X_data = np.concatenate((X_data, data_signal), axis=0)

#             self.X_data_all.append(X_data)
#             self.y_data_all.append(metadata_file['age_months'])
        
#     def on_epoch_end(self):
#         """Updates indexes after each epoch."""

#         # Create new metadata DataFrame with only the current subject IDs
#         if self.metadata_temp is None:
#             self.metadata_temp = self.metadata[self.metadata['code'].isin(self.list_IDs)].reset_index(drop=True)
                               
#         idx_base = np.arange(len(self.metadata_temp))
#         idx_epoch = np.tile(idx_base, self.iter_per_epoch)

#         self.indexes = idx_epoch

#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def generate_data(self, indexes):
#         """Generates data containing batch_size averaged time series.

#         Args:
#         -------
#         list_IDs_temp: list
#             list of label ids to load

#         return: batch of averaged time series
#         """
#         X_data = np.zeros((0, self.n_channels, self.n_timepoints))
#         y_data = []

#         for index in indexes:
#             X = self.create_averaged_epoch(self.X_data_all[index])

#             X_data = np.concatenate((X_data, X), axis=0)
#             y_data.append(self.y_data_all[index])
            
#         return np.swapaxes(X_data,1,2), np.array(y_data).reshape((-1,1))
    
#     def create_averaged_epoch(self,
#                               data_signal):
#         """
#         Function to create averages of self.n_average epochs.
#         Will create one averaged epoch per found unique label from self.n_average random epochs.

#         Args:
#         --------
#         data_signal: numpy array
#             Data from one person as numpy array
#         """
                                               
#         # Create new data collection:
#         X_data = np.zeros((0, self.n_channels, self.n_timepoints))
#         num_epochs = len(data_signal)
                                               
#         if num_epochs >= self.n_average:
#             select = np.random.choice(num_epochs, self.n_average, replace=False)
#             signal_averaged = np.mean(data_signal[select,:,:], axis=0)

#         else:
#             if self.warnings:
#                 print("Found only", num_epochs, " epochs and will take those!")          
#             signal_averaged = np.mean(data_signal[:,:,:], axis=0)
                                                                                              
#         X_data = np.concatenate([X_data, np.expand_dims(signal_averaged, axis=0)], axis=0)
                                    
#         if self.gaussian_noise != 0.0:
#             X_data += np.random.normal(0, self.gaussian_noise, X_data.shape)

#         return X_data


#     def load_signal(self,
#                     filename):
#         """Load EEG signal from one person.

#         Args:
#         -------
#         filename: str
#             filename...

#         return: loaded array
#         """
#         return zarr.open(os.path.join(filename), mode='r')

In [6]:
%%time 
# IDs_train = [152, 18, 11, 435, 617]
# IDs_val = [180, 105, 19, 332]

train_generator_noise = DataGeneratorReduced(list_IDs = IDs_train,
                                             BASE_PATH = PATH_DATA_PROCESSED_DL_REDUCED,
                                             metadata = df_metadata,
                                             n_average = 3,
                                             batch_size = 10,
                                             gaussian_noise=0.01,
                                             iter_per_epoch = 3,
                                             n_timepoints = 501, 
                                             n_channels=30, 
                                             shuffle=True)

val_generator = DataGeneratorReduced(list_IDs = IDs_val,
                                     BASE_PATH = PATH_DATA_PROCESSED_DL_REDUCED,
                                     metadata = df_metadata,
                                     n_average = 3,
                                     batch_size = 10,
                                     n_timepoints = 501,
                                     iter_per_epoch = 3,
                                     n_channels=30,
                                     shuffle=True)

CPU times: user 22.5 s, sys: 22 s, total: 44.5 s
Wall time: 1min 17s


(10, 501, 30)

# Helper functions

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, BatchNormalization, Dense, Conv1D, LeakyReLU, AveragePooling1D, Flatten, Reshape, MaxPooling1D
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
import time

n_timesteps = 501
n_features = 30 
n_outputs = 1

input_shape = (n_timesteps, n_features)

# Testing of different architectures

Below we will test multiple different architectures, most of them as discussed in "Deep learning for time series classification: a review", by Ismail Fawaz et al (2019). Most of them are inspired again on other papers. Refer to the Ismail Fawaz paper for the original papers.

1. Fully-connected NN
2. CNN
3. ResNet
4. Encoder
5. Time-CNN

Other architectures to test:

6. BLSTM-LSTM 
7. InceptionTime


# 1. Fully connected NN

In [9]:
def fully_connected_model():
    """ Returns the fully connected model from Ismail Fawaz et al. (2019). """

    input_layer = keras.layers.Input(input_shape)

    input_layer_flattened = keras.layers.Flatten()(input_layer)

    layer_1 = keras.layers.Dropout(0.1)(input_layer_flattened)
    layer_1 = keras.layers.Dense(500, activation='relu')(layer_1)

    layer_2 = keras.layers.Dropout(0.2)(layer_1)
    layer_2 = keras.layers.Dense(500, activation='relu')(layer_2)

    layer_3 = keras.layers.Dropout(0.2)(layer_2)
    layer_3 = keras.layers.Dense(500, activation='relu')(layer_3)

    output_layer = keras.layers.Dropout(0.3)(layer_3)
    output_layer = keras.layers.Dense(1)(output_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [10]:
model = fully_connected_model()

optimizer = Adadelta(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# 01 seems to be incorrect (makes too many predictions, changed model)
# Fully_connected_regressor_01: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
# Fully_connected_regressor_02: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
output_filename = 'Fully_connected_regressor_03'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=200, min_lr=0.0001, verbose=1)

In [11]:
%%time

epochs = 5000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/5000
83/83 [==============================] - 6s 68ms/step - loss: 868.5831 - root_mean_squared_error: 29.4656 - mean_absolute_error: 27.5962 - val_loss: 638.1409 - val_root_mean_squared_error: 25.2614 - val_mean_absolute_error: 23.1626

Epoch 00001: val_loss improved from inf to 638.14087, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Fully_connected_regressor_03.hdf5
Epoch 2/5000
83/83 [==============================] - 5s 55ms/step - loss: 505.3884 - root_mean_squared_error: 22.4179 - mean_absolute_error: 20.1228 - val_loss: 298.9310 - val_root_mean_squared_error: 17.2896 - val_mean_absolute_error: 14.1719

Epoch 00002: val_loss improved from 638.14087 to 298.93100, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Fully_connected_regressor_03.hdf5
Epoch 3/5000
83/83 [==============================] - 4s 47ms/step - loss: 272.8383 - root_mean_squared_error: 16.5092 - mean_absolute_error: 13

KeyboardInterrupt: 

# 2. CNN

In [18]:
def cnn_model():
    """ Returns the CNN (FCN) model from Ismail Fawaz et al. (2019). """

    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3, padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(1)(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model 

In [21]:
model = cnn_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# CNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'CNN_regressor_03'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=250, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [22]:
%%time

epochs = 2000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/2000
84/84 [==============================] - ETA: 0s - loss: nan - root_mean_squared_error: nan - mean_absolute_error: nan

KeyboardInterrupt: 

# 3. ResNet

In [22]:
def resnet_model():
    """ Returns the ResNet model from Ismail Fawaz et al. (2019). """
    n_feature_maps = 64

    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # BLOCK 2

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # BLOCK 3

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # no need to expand channels because they are equal
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # FINAL

    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

    output_layer = keras.layers.Dense(1)(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [23]:
model = resnet_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# ResNet_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'ResNet_regressor_02'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=250, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [24]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
143/143 [==============================] - 95s 646ms/step - loss: 189.6503 - root_mean_squared_error: 13.4258 - mean_absolute_error: 11.0235 - val_loss: 1908.8988 - val_root_mean_squared_error: 43.6909 - val_mean_absolute_error: 35.3563

Epoch 00001: val_loss improved from inf to 1908.89880, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/ResNet_regressor_01.hdf5
Epoch 2/1500
143/143 [==============================] - 89s 619ms/step - loss: 109.5164 - root_mean_squared_error: 10.4630 - mean_absolute_error: 8.7816 - val_loss: 142.7589 - val_root_mean_squared_error: 11.9482 - val_mean_absolute_error: 9.5680

Epoch 00002: val_loss improved from 1908.89880 to 142.75887, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/ResNet_regressor_01.hdf5
Epoch 3/1500
143/143 [==============================] - 90s 630ms/step - loss: 104.9589 - root_mean_squared_error: 10.2428 - mean_absolute_error: 8.6765 

143/143 [==============================] - 91s 635ms/step - loss: 62.4524 - root_mean_squared_error: 7.8951 - mean_absolute_error: 6.3252 - val_loss: 68.3334 - val_root_mean_squared_error: 8.2664 - val_mean_absolute_error: 6.7822

Epoch 00051: val_loss did not improve from 63.92428
Epoch 52/1500
143/143 [==============================] - 91s 635ms/step - loss: 68.8643 - root_mean_squared_error: 8.2923 - mean_absolute_error: 6.8354 - val_loss: 74.9792 - val_root_mean_squared_error: 8.6591 - val_mean_absolute_error: 6.7676

Epoch 00052: val_loss did not improve from 63.92428
Epoch 53/1500
143/143 [==============================] - 90s 631ms/step - loss: 65.4128 - root_mean_squared_error: 8.0848 - mean_absolute_error: 6.5465 - val_loss: 70.1540 - val_root_mean_squared_error: 8.3758 - val_mean_absolute_error: 6.7421

Epoch 00053: val_loss did not improve from 63.92428
Epoch 54/1500
143/143 [==============================] - 92s 640ms/step - loss: 62.5797 - root_mean_squared_error: 7.9066 -

143/143 [==============================] - 93s 647ms/step - loss: 60.9838 - root_mean_squared_error: 7.7936 - mean_absolute_error: 6.3301 - val_loss: 74.6349 - val_root_mean_squared_error: 8.6391 - val_mean_absolute_error: 6.9218

Epoch 00104: val_loss did not improve from 56.65411
Epoch 105/1500
143/143 [==============================] - 93s 651ms/step - loss: 62.2411 - root_mean_squared_error: 7.8813 - mean_absolute_error: 6.3707 - val_loss: 122.1213 - val_root_mean_squared_error: 11.0509 - val_mean_absolute_error: 9.3274

Epoch 00105: val_loss did not improve from 56.65411
Epoch 106/1500
143/143 [==============================] - 93s 652ms/step - loss: 62.3770 - root_mean_squared_error: 7.8961 - mean_absolute_error: 6.4425 - val_loss: 82.1519 - val_root_mean_squared_error: 9.0638 - val_mean_absolute_error: 7.4407

Epoch 00106: val_loss did not improve from 56.65411
Epoch 107/1500
143/143 [==============================] - 92s 643ms/step - loss: 58.2403 - root_mean_squared_error: 7.6


Epoch 00158: val_loss did not improve from 51.60717
Epoch 159/1500
143/143 [==============================] - 91s 638ms/step - loss: 54.2945 - root_mean_squared_error: 7.3653 - mean_absolute_error: 6.0166 - val_loss: 73.6091 - val_root_mean_squared_error: 8.5796 - val_mean_absolute_error: 6.9942

Epoch 00159: val_loss did not improve from 51.60717
Epoch 160/1500
143/143 [==============================] - 91s 634ms/step - loss: 57.3901 - root_mean_squared_error: 7.5715 - mean_absolute_error: 6.1807 - val_loss: 84.3955 - val_root_mean_squared_error: 9.1867 - val_mean_absolute_error: 7.2678

Epoch 00160: val_loss did not improve from 51.60717
Epoch 161/1500
143/143 [==============================] - 90s 631ms/step - loss: 62.8495 - root_mean_squared_error: 7.9244 - mean_absolute_error: 6.3541 - val_loss: 102.1664 - val_root_mean_squared_error: 10.1077 - val_mean_absolute_error: 7.8854

Epoch 00161: val_loss did not improve from 51.60717
Epoch 162/1500
143/143 [===========================

143/143 [==============================] - 91s 639ms/step - loss: 52.3391 - root_mean_squared_error: 7.2312 - mean_absolute_error: 5.9196 - val_loss: 59.0580 - val_root_mean_squared_error: 7.6849 - val_mean_absolute_error: 6.1612

Epoch 00186: val_loss did not improve from 51.60717
Epoch 187/1500
143/143 [==============================] - 90s 630ms/step - loss: 57.1198 - root_mean_squared_error: 7.5562 - mean_absolute_error: 6.1553 - val_loss: 57.7867 - val_root_mean_squared_error: 7.6018 - val_mean_absolute_error: 6.0612

Epoch 00187: val_loss did not improve from 51.60717
Epoch 188/1500
143/143 [==============================] - 90s 628ms/step - loss: 53.4742 - root_mean_squared_error: 7.3093 - mean_absolute_error: 5.8384 - val_loss: 72.1455 - val_root_mean_squared_error: 8.4938 - val_mean_absolute_error: 6.8769

Epoch 00188: val_loss did not improve from 51.60717
Epoch 189/1500
143/143 [==============================] - 91s 635ms/step - loss: 53.1975 - root_mean_squared_error: 7.288

...


Epoch 00268: val_loss did not improve from 51.60717
Epoch 269/1500
143/143 [==============================] - 90s 632ms/step - loss: 49.6948 - root_mean_squared_error: 7.0430 - mean_absolute_error: 5.5605 - val_loss: 66.4047 - val_root_mean_squared_error: 8.1489 - val_mean_absolute_error: 6.6196

Epoch 00269: val_loss did not improve from 51.60717
Epoch 270/1500
143/143 [==============================] - 90s 630ms/step - loss: 48.8633 - root_mean_squared_error: 6.9869 - mean_absolute_error: 5.5902 - val_loss: 71.9904 - val_root_mean_squared_error: 8.4847 - val_mean_absolute_error: 6.6699

Epoch 00270: val_loss did not improve from 51.60717
Epoch 271/1500
143/143 [==============================] - 91s 635ms/step - loss: 51.0616 - root_mean_squared_error: 7.1275 - mean_absolute_error: 5.7137 - val_loss: 68.7816 - val_root_mean_squared_error: 8.2935 - val_mean_absolute_error: 6.5994

Epoch 00271: val_loss did not improve from 51.60717
Epoch 272/1500
143/143 [=============================

143/143 [==============================] - 90s 632ms/step - loss: 52.0311 - root_mean_squared_error: 7.2117 - mean_absolute_error: 5.8255 - val_loss: 68.0905 - val_root_mean_squared_error: 8.2517 - val_mean_absolute_error: 6.5943

Epoch 00296: val_loss did not improve from 51.60717
Epoch 297/1500
143/143 [==============================] - 91s 639ms/step - loss: 52.5157 - root_mean_squared_error: 7.2415 - mean_absolute_error: 5.8090 - val_loss: 68.0638 - val_root_mean_squared_error: 8.2501 - val_mean_absolute_error: 6.5434

Epoch 00297: val_loss did not improve from 51.60717
Epoch 298/1500
143/143 [==============================] - 92s 644ms/step - loss: 48.5344 - root_mean_squared_error: 6.9594 - mean_absolute_error: 5.5647 - val_loss: 57.6487 - val_root_mean_squared_error: 7.5927 - val_mean_absolute_error: 6.0853

Epoch 00298: val_loss did not improve from 51.60717
Epoch 299/1500
143/143 [==============================] - 91s 633ms/step - loss: 49.6067 - root_mean_squared_error: 7.040


Epoch 00323: val_loss did not improve from 51.60717
Epoch 324/1500
143/143 [==============================] - 91s 635ms/step - loss: 50.7925 - root_mean_squared_error: 7.1204 - mean_absolute_error: 5.7234 - val_loss: 58.8398 - val_root_mean_squared_error: 7.6707 - val_mean_absolute_error: 6.3481

Epoch 00324: val_loss did not improve from 51.60717
Epoch 325/1500
143/143 [==============================] - 92s 642ms/step - loss: 49.7355 - root_mean_squared_error: 7.0500 - mean_absolute_error: 5.7444 - val_loss: 61.8390 - val_root_mean_squared_error: 7.8638 - val_mean_absolute_error: 6.3203

Epoch 00325: val_loss did not improve from 51.60717
Epoch 326/1500
143/143 [==============================] - 91s 638ms/step - loss: 49.4509 - root_mean_squared_error: 7.0230 - mean_absolute_error: 5.6665 - val_loss: 59.2194 - val_root_mean_squared_error: 7.6954 - val_mean_absolute_error: 6.2665

Epoch 00326: val_loss did not improve from 51.60717
Epoch 327/1500
143/143 [=============================

143/143 [==============================] - 92s 642ms/step - loss: 46.9661 - root_mean_squared_error: 6.8459 - mean_absolute_error: 5.5580 - val_loss: 60.7570 - val_root_mean_squared_error: 7.7947 - val_mean_absolute_error: 6.1445

Epoch 00351: val_loss did not improve from 51.60717
Epoch 352/1500
143/143 [==============================] - 91s 638ms/step - loss: 47.3128 - root_mean_squared_error: 6.8743 - mean_absolute_error: 5.5568 - val_loss: 64.8969 - val_root_mean_squared_error: 8.0559 - val_mean_absolute_error: 6.6133

Epoch 00352: val_loss did not improve from 51.60717
Epoch 353/1500
143/143 [==============================] - 92s 647ms/step - loss: 49.4599 - root_mean_squared_error: 7.0276 - mean_absolute_error: 5.6301 - val_loss: 73.7757 - val_root_mean_squared_error: 8.5893 - val_mean_absolute_error: 6.8152

Epoch 00353: val_loss did not improve from 51.60717
Epoch 354/1500
143/143 [==============================] - 94s 656ms/step - loss: 49.8391 - root_mean_squared_error: 7.055


Epoch 00378: val_loss did not improve from 51.60717

Epoch 00378: ReduceLROnPlateau reducing learning rate to 0.0003124999930150807.
Epoch 00378: early stopping
CPU times: user 14h 37min 6s, sys: 6h 8min 32s, total: 20h 45min 38s
Wall time: 9h 38min 13s


# 4. Encoder

In [27]:
import tensorflow_addons as tfa

def encoder_model():
    """ Returns the Encoder model from Ismail Fawaz et al. (2019). """
    input_layer = keras.layers.Input(input_shape)

    # conv block -1
    conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(input_layer)
    conv1 = tfa.layers.InstanceNormalization()(conv1)
    conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
    conv1 = keras.layers.Dropout(rate=0.2)(conv1)
    conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # conv block -2
    conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(conv1)
    conv2 = tfa.layers.InstanceNormalization()(conv2)
    conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
    conv2 = keras.layers.Dropout(rate=0.2)(conv2)
    conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
    # conv block -3
    conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv2)
    conv3 = tfa.layers.InstanceNormalization()(conv3)
    conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
    conv3 = keras.layers.Dropout(rate=0.2)(conv3)
    # split for attention
    attention_data = keras.layers.Lambda(lambda x: x[:,:,:256])(conv3)
    attention_softmax = keras.layers.Lambda(lambda x: x[:,:,256:])(conv3)
    # attention mechanism
    attention_softmax = keras.layers.Softmax()(attention_softmax)
    multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
    # last layer
    dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
    dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
    # output layer
    flatten_layer = keras.layers.Flatten()(dense_layer)
    output_layer = keras.layers.Dense(1)(flatten_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

/Users/bbruns/anaconda3/envs/mne/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [28]:
model = encoder_model()

optimizer = Adam(learning_rate=0.00001)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# Encoder_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01 (LR = 0.0001, no reduction)
output_filename = 'Encoder_regressor_02'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)

In [29]:
%%time

epochs = 100

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer])

Epoch 1/100
143/143 [==============================] - 97s 669ms/step - loss: 772.3317 - root_mean_squared_error: 27.7892 - mean_absolute_error: 25.8278 - val_loss: 711.6334 - val_root_mean_squared_error: 26.6765 - val_mean_absolute_error: 24.4745

Epoch 00001: val_loss improved from inf to 711.63336, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_01.hdf5
Epoch 2/100
143/143 [==============================] - 93s 654ms/step - loss: 585.1881 - root_mean_squared_error: 24.1294 - mean_absolute_error: 21.6805 - val_loss: 317.0688 - val_root_mean_squared_error: 17.8064 - val_mean_absolute_error: 15.0762

Epoch 00002: val_loss improved from 711.63336 to 317.06882, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_01.hdf5
Epoch 3/100
143/143 [==============================] - 92s 641ms/step - loss: 192.9793 - root_mean_squared_error: 13.8633 - mean_absolute_error: 11.2211 -

143/143 [==============================] - 131s 916ms/step - loss: 63.3927 - root_mean_squared_error: 7.9580 - mean_absolute_error: 6.3827 - val_loss: 83.8684 - val_root_mean_squared_error: 9.1580 - val_mean_absolute_error: 7.2467

Epoch 00052: val_loss did not improve from 73.91944
Epoch 53/100
143/143 [==============================] - 142s 993ms/step - loss: 62.4662 - root_mean_squared_error: 7.9010 - mean_absolute_error: 6.3508 - val_loss: 84.5560 - val_root_mean_squared_error: 9.1954 - val_mean_absolute_error: 7.0287

Epoch 00053: val_loss did not improve from 73.91944
Epoch 54/100
143/143 [==============================] - 143s 997ms/step - loss: 61.8955 - root_mean_squared_error: 7.8632 - mean_absolute_error: 6.2992 - val_loss: 83.9788 - val_root_mean_squared_error: 9.1640 - val_mean_absolute_error: 7.1886

Epoch 00054: val_loss did not improve from 73.91944
Epoch 55/100
143/143 [==============================] - 175s 1s/step - loss: 58.3629 - root_mean_squared_error: 7.6310 - m

# 5. Time-CNN

In [33]:
# https://github.com/hfawaz/dl-4-tsc/blob/master/classifiers/cnn.py

def timecnn_model():
    """ Returns the Time-CNN model from Ismail Fawaz et al. (2019). """
    
    padding = 'valid'
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=6,kernel_size=7,padding=padding,activation='sigmoid')(input_layer)
    conv1 = keras.layers.AveragePooling1D(pool_size=3)(conv1)

    conv2 = keras.layers.Conv1D(filters=12,kernel_size=7,padding=padding,activation='sigmoid')(conv1)
    conv2 = keras.layers.AveragePooling1D(pool_size=3)(conv2)

    flatten_layer = keras.layers.Flatten()(conv2)

    output_layer = keras.layers.Dense(units=1)(flatten_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [34]:
model = cnn_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# TimeCNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'TimeCNN_regressor_02'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=250, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [35]:
%%time

epochs = 2000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/2000
143/143 [==============================] - 93s 642ms/step - loss: 321.7530 - root_mean_squared_error: 17.4269 - mean_absolute_error: 14.5173 - val_loss: 160.2513 - val_root_mean_squared_error: 12.6590 - val_mean_absolute_error: 10.2964

Epoch 00001: val_loss improved from inf to 160.25128, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/TimeCNN_regressor_01.hdf5
Epoch 2/2000
143/143 [==============================] - 89s 623ms/step - loss: 99.1723 - root_mean_squared_error: 9.9552 - mean_absolute_error: 8.2079 - val_loss: 187.8685 - val_root_mean_squared_error: 13.7065 - val_mean_absolute_error: 10.9276

Epoch 00002: val_loss did not improve from 160.25128
Epoch 3/2000
143/143 [==============================] - 89s 622ms/step - loss: 105.3432 - root_mean_squared_error: 10.2628 - mean_absolute_error: 8.5573 - val_loss: 121.3760 - val_root_mean_squared_error: 11.0171 - val_mean_absolute_error: 9.1967

Epoch 00003: val_loss improved 


Epoch 00024: val_loss did not improve from 83.43713
Epoch 25/2000
143/143 [==============================] - 86s 603ms/step - loss: 84.7795 - root_mean_squared_error: 9.2056 - mean_absolute_error: 7.6125 - val_loss: 95.3176 - val_root_mean_squared_error: 9.7631 - val_mean_absolute_error: 7.7439

Epoch 00025: val_loss did not improve from 83.43713
Epoch 26/2000
143/143 [==============================] - 83s 581ms/step - loss: 89.2550 - root_mean_squared_error: 9.4411 - mean_absolute_error: 7.7936 - val_loss: 89.3617 - val_root_mean_squared_error: 9.4531 - val_mean_absolute_error: 7.7796

Epoch 00026: val_loss did not improve from 83.43713
Epoch 27/2000
143/143 [==============================] - 83s 584ms/step - loss: 84.8994 - root_mean_squared_error: 9.2108 - mean_absolute_error: 7.6540 - val_loss: 100.0338 - val_root_mean_squared_error: 10.0017 - val_mean_absolute_error: 8.5045

Epoch 00027: val_loss did not improve from 83.43713
Epoch 28/2000
143/143 [==============================]


Epoch 00076: val_loss did not improve from 63.82006
Epoch 77/2000
143/143 [==============================] - 90s 628ms/step - loss: 61.4023 - root_mean_squared_error: 7.8217 - mean_absolute_error: 6.2745 - val_loss: 91.0890 - val_root_mean_squared_error: 9.5441 - val_mean_absolute_error: 7.3644

Epoch 00077: val_loss did not improve from 63.82006
Epoch 78/2000
143/143 [==============================] - 89s 621ms/step - loss: 68.1493 - root_mean_squared_error: 8.2511 - mean_absolute_error: 6.6158 - val_loss: 62.1471 - val_root_mean_squared_error: 7.8833 - val_mean_absolute_error: 6.4276

Epoch 00078: val_loss improved from 63.82006 to 62.14713, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/TimeCNN_regressor_01.hdf5
Epoch 79/2000
143/143 [==============================] - 88s 618ms/step - loss: 66.1399 - root_mean_squared_error: 8.1301 - mean_absolute_error: 6.6317 - val_loss: 75.2805 - val_root_mean_squared_error: 8.6764 - val_mean_absolute_


Epoch 00130: val_loss did not improve from 60.16933
Epoch 131/2000
143/143 [==============================] - 89s 625ms/step - loss: 61.6942 - root_mean_squared_error: 7.8531 - mean_absolute_error: 6.3671 - val_loss: 76.5305 - val_root_mean_squared_error: 8.7482 - val_mean_absolute_error: 7.2403

Epoch 00131: val_loss did not improve from 60.16933
Epoch 132/2000
143/143 [==============================] - 88s 617ms/step - loss: 63.6344 - root_mean_squared_error: 7.9750 - mean_absolute_error: 6.5020 - val_loss: 64.0276 - val_root_mean_squared_error: 8.0017 - val_mean_absolute_error: 6.5223

Epoch 00132: val_loss did not improve from 60.16933
Epoch 133/2000
143/143 [==============================] - 88s 618ms/step - loss: 58.2803 - root_mean_squared_error: 7.6308 - mean_absolute_error: 6.0975 - val_loss: 71.5432 - val_root_mean_squared_error: 8.4583 - val_mean_absolute_error: 7.1619

Epoch 00133: val_loss did not improve from 60.16933
Epoch 134/2000
143/143 [=============================

143/143 [==============================] - 84s 588ms/step - loss: 56.8468 - root_mean_squared_error: 7.5335 - mean_absolute_error: 6.0720 - val_loss: 67.4329 - val_root_mean_squared_error: 8.2118 - val_mean_absolute_error: 6.3400

Epoch 00158: val_loss did not improve from 58.89191
Epoch 159/2000
143/143 [==============================] - 86s 604ms/step - loss: 61.2566 - root_mean_squared_error: 7.8251 - mean_absolute_error: 6.4088 - val_loss: 110.2732 - val_root_mean_squared_error: 10.5011 - val_mean_absolute_error: 8.7393

Epoch 00159: val_loss did not improve from 58.89191
Epoch 160/2000
143/143 [==============================] - 86s 600ms/step - loss: 56.1984 - root_mean_squared_error: 7.4957 - mean_absolute_error: 5.9987 - val_loss: 75.5192 - val_root_mean_squared_error: 8.6902 - val_mean_absolute_error: 6.9132

Epoch 00160: val_loss did not improve from 58.89191
Epoch 161/2000
143/143 [==============================] - 86s 604ms/step - loss: 64.0623 - root_mean_squared_error: 7.9

...

143/143 [==============================] - 93s 651ms/step - loss: 46.5473 - root_mean_squared_error: 6.8148 - mean_absolute_error: 5.5335 - val_loss: 65.3938 - val_root_mean_squared_error: 8.0866 - val_mean_absolute_error: 6.4036

Epoch 00760: val_loss did not improve from 48.57585
Epoch 761/2000
143/143 [==============================] - 91s 639ms/step - loss: 51.9924 - root_mean_squared_error: 7.2091 - mean_absolute_error: 5.7113 - val_loss: 59.9219 - val_root_mean_squared_error: 7.7409 - val_mean_absolute_error: 6.0618

Epoch 00761: val_loss did not improve from 48.57585
Epoch 762/2000
143/143 [==============================] - 90s 632ms/step - loss: 47.6922 - root_mean_squared_error: 6.9001 - mean_absolute_error: 5.5959 - val_loss: 63.3959 - val_root_mean_squared_error: 7.9622 - val_mean_absolute_error: 6.2361

Epoch 00762: val_loss did not improve from 48.57585
Epoch 763/2000
143/143 [==============================] - 90s 631ms/step - loss: 49.0940 - root_mean_squared_error: 7.004


Epoch 00787: val_loss did not improve from 48.57585
Epoch 788/2000
143/143 [==============================] - 90s 633ms/step - loss: 42.3058 - root_mean_squared_error: 6.5030 - mean_absolute_error: 5.1744 - val_loss: 57.1928 - val_root_mean_squared_error: 7.5626 - val_mean_absolute_error: 6.1126

Epoch 00788: val_loss did not improve from 48.57585
Epoch 789/2000
143/143 [==============================] - 91s 635ms/step - loss: 46.7806 - root_mean_squared_error: 6.8373 - mean_absolute_error: 5.6046 - val_loss: 71.4061 - val_root_mean_squared_error: 8.4502 - val_mean_absolute_error: 6.6661

Epoch 00789: val_loss did not improve from 48.57585
Epoch 790/2000
143/143 [==============================] - 90s 626ms/step - loss: 43.1648 - root_mean_squared_error: 6.5648 - mean_absolute_error: 5.3594 - val_loss: 69.0603 - val_root_mean_squared_error: 8.3103 - val_mean_absolute_error: 6.6066

Epoch 00790: val_loss did not improve from 48.57585
Epoch 791/2000
143/143 [=============================


Epoch 00842: val_loss did not improve from 48.57585
Epoch 843/2000
143/143 [==============================] - 83s 583ms/step - loss: 46.0476 - root_mean_squared_error: 6.7839 - mean_absolute_error: 5.5289 - val_loss: 69.3073 - val_root_mean_squared_error: 8.3251 - val_mean_absolute_error: 6.5033

Epoch 00843: val_loss did not improve from 48.57585
Epoch 844/2000
143/143 [==============================] - 83s 581ms/step - loss: 48.2691 - root_mean_squared_error: 6.9458 - mean_absolute_error: 5.6355 - val_loss: 58.6813 - val_root_mean_squared_error: 7.6604 - val_mean_absolute_error: 6.1438

Epoch 00844: val_loss did not improve from 48.57585
Epoch 845/2000
143/143 [==============================] - 83s 583ms/step - loss: 47.3390 - root_mean_squared_error: 6.8692 - mean_absolute_error: 5.5207 - val_loss: 70.1871 - val_root_mean_squared_error: 8.3778 - val_mean_absolute_error: 6.4513

Epoch 00845: val_loss did not improve from 48.57585
Epoch 846/2000
143/143 [=============================


Epoch 00897: val_loss did not improve from 48.57585
Epoch 898/2000
143/143 [==============================] - 75s 528ms/step - loss: 44.7182 - root_mean_squared_error: 6.6848 - mean_absolute_error: 5.3195 - val_loss: 57.5303 - val_root_mean_squared_error: 7.5849 - val_mean_absolute_error: 6.1032

Epoch 00898: val_loss did not improve from 48.57585
Epoch 899/2000
143/143 [==============================] - 75s 523ms/step - loss: 46.4311 - root_mean_squared_error: 6.8112 - mean_absolute_error: 5.4527 - val_loss: 69.5810 - val_root_mean_squared_error: 8.3415 - val_mean_absolute_error: 6.7588

Epoch 00899: val_loss did not improve from 48.57585
Epoch 900/2000
143/143 [==============================] - 75s 526ms/step - loss: 47.6541 - root_mean_squared_error: 6.9011 - mean_absolute_error: 5.5875 - val_loss: 64.2698 - val_root_mean_squared_error: 8.0168 - val_mean_absolute_error: 6.3858

Epoch 00900: val_loss did not improve from 48.57585
Epoch 901/2000
143/143 [=============================


Epoch 00952: val_loss did not improve from 48.57585
Epoch 953/2000
143/143 [==============================] - 77s 542ms/step - loss: 45.1069 - root_mean_squared_error: 6.7104 - mean_absolute_error: 5.3789 - val_loss: 53.8673 - val_root_mean_squared_error: 7.3394 - val_mean_absolute_error: 6.0017

Epoch 00953: val_loss did not improve from 48.57585
Epoch 954/2000
143/143 [==============================] - 81s 566ms/step - loss: 44.6838 - root_mean_squared_error: 6.6734 - mean_absolute_error: 5.3476 - val_loss: 72.3202 - val_root_mean_squared_error: 8.5041 - val_mean_absolute_error: 6.9326

Epoch 00954: val_loss did not improve from 48.57585
Epoch 955/2000
143/143 [==============================] - 83s 583ms/step - loss: 44.4511 - root_mean_squared_error: 6.6664 - mean_absolute_error: 5.4114 - val_loss: 57.7540 - val_root_mean_squared_error: 7.5996 - val_mean_absolute_error: 5.8980

Epoch 00955: val_loss did not improve from 48.57585
Epoch 956/2000
143/143 [=============================

# 6. BLSTM-LSTM

In [16]:
def blstm_lstm_model():
    """ Returns the BLSTM-LSTM model from Kaushik et al. (2019). """

    # MARK: This model compresses too much in the last phase, check if possible to improve.

    model = keras.Sequential()

    # BLSTM layer
    model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(.2))
    model.add(BatchNormalization())

    # LSTM layer
    model.add(LSTM(128, return_sequences=True))
    model.add(BatchNormalization())

    # LSTM layer
    model.add(LSTM(64, return_sequences=False))
    model.add(BatchNormalization())

    # Fully connected layer
    model.add(Dense(32))

    model.add(Dense(n_outputs))

    return model 

In [20]:
model = blstm_lstm_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# BLSTM_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'BLSTM_regressor_01'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=250, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 501, 512)          587776    
_________________________________________________________________
dropout_6 (Dropout)          (None, 501, 512)          0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 501, 512)          2048      
_________________________________________________________________
lstm_7 (LSTM)                (None, 501, 128)          328192    
_________________________________________________________________
batch_normalization_7 (Batch (None, 501, 128)          512       
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
batch_normalization_8 (Batch (None, 64)               

In [19]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
83/83 [==============================] - 81s 973ms/step - loss: 226.7901 - root_mean_squared_error: 15.0596 - mean_absolute_error: 12.0603 - val_loss: 277.5875 - val_root_mean_squared_error: 16.6610 - val_mean_absolute_error: 14.4406

Epoch 00001: val_loss improved from inf to 277.58746, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/BLSTM_regressor_01.hdf5
Epoch 2/1500
61/83 [=====================>........] - ETA: 24s - loss: 131.2852 - root_mean_squared_error: 11.4580 - mean_absolute_error: 9.6893

KeyboardInterrupt: 

# 7. InceptionTime

In [14]:
class Regressor_Inception:

    def __init__(self, output_directory, input_shape, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'inception_model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = tf.keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = tf.keras.layers.Concatenate(axis=2)(conv_list)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = tf.keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

        x = tf.keras.layers.Add()([shortcut_y, out_tensor])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape):
        input_layer = tf.keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        pooling_layer = tf.keras.layers.AveragePooling1D(pool_size=50)(x)
        flat_layer = tf.keras.layers.Flatten()(pooling_layer)
        dense_layer = tf.keras.layers.Dense(128, activation='relu')(flat_layer)
        output_layer = tf.keras.layers.Dense(1)(dense_layer)

        model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

        return model

In [ ]:
model = Regressor_Inception(PATH_MODELS, input_shape, verbose=True).model

optimizer = Adam(learning_rate=0.01)   
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# 'Inception_regressor_01' (n_average = 40, gaussian_noise = 0.01, MAE)
# 'Inception_regressor_02' (n_average = 1, gaussian_noise = 0.01, MAE)
# 'Inception_regressor_03' (n_average = 40, gaussian_noise = 0.01, MSE)
# 'Inception_regressor_04' (n_average = 1, gaussian_noise = 0.01, MSE)
# 'Inception_regressor_05' (n_average = 100, gaussian_noise = 0.01, MAE)
output_filename = 'Inception_regressor_05'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=100, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, min_lr=0.0001, verbose=1)

In [96]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs,
                    callbacks = [checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
143/143 [==============================] - 249s 2s/step - loss: 14.8795 - root_mean_squared_error: 21.7512 - mean_absolute_error: 14.8795 - val_loss: 14.4589 - val_root_mean_squared_error: 18.1789 - val_mean_absolute_error: 14.4589

Epoch 00001: val_loss improved from inf to 14.45891, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 2/1500
143/143 [==============================] - 238s 2s/step - loss: 8.9878 - root_mean_squared_error: 11.2585 - mean_absolute_error: 8.9878 - val_loss: 8.4816 - val_root_mean_squared_error: 10.7197 - val_mean_absolute_error: 8.4816

Epoch 00002: val_loss improved from 14.45891 to 8.48164, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 3/1500
143/143 [==============================] - 241s 2s/step - loss: 7.9297 - root_mean_squared_error: 9.8073 - mean_absolute_error: 7.9297 - val_loss: 8.62


Epoch 00024: val_loss did not improve from 5.82968
Epoch 25/1500
143/143 [==============================] - 249s 2s/step - loss: 5.9342 - root_mean_squared_error: 7.4770 - mean_absolute_error: 5.9342 - val_loss: 5.8190 - val_root_mean_squared_error: 7.3576 - val_mean_absolute_error: 5.8190

Epoch 00025: val_loss improved from 5.82968 to 5.81904, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 26/1500
143/143 [==============================] - 244s 2s/step - loss: 5.8491 - root_mean_squared_error: 7.5007 - mean_absolute_error: 5.8491 - val_loss: 6.2931 - val_root_mean_squared_error: 7.9237 - val_mean_absolute_error: 6.2931

Epoch 00026: val_loss did not improve from 5.81904
Epoch 27/1500
143/143 [==============================] - 234s 2s/step - loss: 5.7424 - root_mean_squared_error: 7.2581 - mean_absolute_error: 5.7424 - val_loss: 5.7668 - val_root_mean_squared_error: 7.2636 - val_mean_absolute_error: 5.7668


143/143 [==============================] - 316s 2s/step - loss: 5.1042 - root_mean_squared_error: 6.6133 - mean_absolute_error: 5.1042 - val_loss: 5.6670 - val_root_mean_squared_error: 7.2707 - val_mean_absolute_error: 5.6670

Epoch 00078: val_loss did not improve from 5.23083
Epoch 79/1500
143/143 [==============================] - 321s 2s/step - loss: 4.9956 - root_mean_squared_error: 6.3485 - mean_absolute_error: 4.9956 - val_loss: 5.4951 - val_root_mean_squared_error: 7.2871 - val_mean_absolute_error: 5.4951

Epoch 00079: val_loss did not improve from 5.23083
Epoch 80/1500
143/143 [==============================] - 324s 2s/step - loss: 5.1040 - root_mean_squared_error: 6.4311 - mean_absolute_error: 5.1040 - val_loss: 5.6553 - val_root_mean_squared_error: 7.0254 - val_mean_absolute_error: 5.6553

Epoch 00080: val_loss did not improve from 5.23083
Epoch 81/1500
143/143 [==============================] - 324s 2s/step - loss: 4.8514 - root_mean_squared_error: 6.2461 - mean_absolute_err

143/143 [==============================] - 303s 2s/step - loss: 4.9634 - root_mean_squared_error: 6.4851 - mean_absolute_error: 4.9634 - val_loss: 6.0805 - val_root_mean_squared_error: 7.7360 - val_mean_absolute_error: 6.0805

Epoch 00106: val_loss did not improve from 5.23083
Epoch 107/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9089 - root_mean_squared_error: 6.3690 - mean_absolute_error: 4.9089 - val_loss: 5.7419 - val_root_mean_squared_error: 7.3350 - val_mean_absolute_error: 5.7419

Epoch 00107: val_loss did not improve from 5.23083
Epoch 108/1500
143/143 [==============================] - 324s 2s/step - loss: 4.6513 - root_mean_squared_error: 5.9217 - mean_absolute_error: 4.6513 - val_loss: 6.1858 - val_root_mean_squared_error: 7.8471 - val_mean_absolute_error: 6.1858

Epoch 00108: val_loss did not improve from 5.23083
Epoch 109/1500
143/143 [==============================] - 321s 2s/step - loss: 5.0534 - root_mean_squared_error: 6.3555 - mean_absolute_


Epoch 00133: val_loss did not improve from 5.20694
Epoch 134/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9270 - root_mean_squared_error: 6.2901 - mean_absolute_error: 4.9270 - val_loss: 5.5962 - val_root_mean_squared_error: 7.2905 - val_mean_absolute_error: 5.5962

Epoch 00134: val_loss did not improve from 5.20694
Epoch 135/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8693 - root_mean_squared_error: 6.1092 - mean_absolute_error: 4.8693 - val_loss: 5.6170 - val_root_mean_squared_error: 7.3019 - val_mean_absolute_error: 5.6170

Epoch 00135: val_loss did not improve from 5.20694
Epoch 136/1500
143/143 [==============================] - 323s 2s/step - loss: 4.7565 - root_mean_squared_error: 6.1799 - mean_absolute_error: 4.7565 - val_loss: 5.7584 - val_root_mean_squared_error: 7.4199 - val_mean_absolute_error: 5.7584

Epoch 00136: val_loss did not improve from 5.20694
Epoch 137/1500
143/143 [==============================] - 323s 2s/ste

143/143 [==============================] - 323s 2s/step - loss: 4.6652 - root_mean_squared_error: 6.0196 - mean_absolute_error: 4.6652 - val_loss: 6.1782 - val_root_mean_squared_error: 7.6756 - val_mean_absolute_error: 6.1782

Epoch 00189: val_loss did not improve from 5.04051
Epoch 190/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8163 - root_mean_squared_error: 6.2274 - mean_absolute_error: 4.8163 - val_loss: 5.3501 - val_root_mean_squared_error: 6.8194 - val_mean_absolute_error: 5.3501

Epoch 00190: val_loss did not improve from 5.04051
Epoch 191/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8694 - root_mean_squared_error: 6.2544 - mean_absolute_error: 4.8694 - val_loss: 5.5560 - val_root_mean_squared_error: 7.1667 - val_mean_absolute_error: 5.5560

Epoch 00191: val_loss did not improve from 5.04051
Epoch 192/1500
143/143 [==============================] - 324s 2s/step - loss: 4.8435 - root_mean_squared_error: 6.1235 - mean_absolute_


Epoch 00216: val_loss did not improve from 4.69455
Epoch 217/1500
143/143 [==============================] - 293s 2s/step - loss: 4.8800 - root_mean_squared_error: 6.2364 - mean_absolute_error: 4.8800 - val_loss: 5.8352 - val_root_mean_squared_error: 7.3876 - val_mean_absolute_error: 5.8352

Epoch 00217: val_loss did not improve from 4.69455
Epoch 218/1500
143/143 [==============================] - 284s 2s/step - loss: 4.6264 - root_mean_squared_error: 5.8839 - mean_absolute_error: 4.6264 - val_loss: 5.5292 - val_root_mean_squared_error: 7.0442 - val_mean_absolute_error: 5.5292

Epoch 00218: val_loss did not improve from 4.69455
Epoch 219/1500
143/143 [==============================] - 311s 2s/step - loss: 5.0450 - root_mean_squared_error: 6.4782 - mean_absolute_error: 5.0450 - val_loss: 5.7673 - val_root_mean_squared_error: 7.2827 - val_mean_absolute_error: 5.7673

Epoch 00219: val_loss did not improve from 4.69455
Epoch 220/1500
143/143 [==============================] - 276s 2s/ste


Epoch 00244: val_loss did not improve from 4.69455
Epoch 245/1500
143/143 [==============================] - 1083s 8s/step - loss: 4.8373 - root_mean_squared_error: 6.3008 - mean_absolute_error: 4.8373 - val_loss: 5.7162 - val_root_mean_squared_error: 7.2399 - val_mean_absolute_error: 5.7162

Epoch 00245: val_loss did not improve from 4.69455
Epoch 246/1500
143/143 [==============================] - 854s 6s/step - loss: 4.7068 - root_mean_squared_error: 6.0775 - mean_absolute_error: 4.7068 - val_loss: 5.7267 - val_root_mean_squared_error: 7.3911 - val_mean_absolute_error: 5.7267

Epoch 00246: val_loss did not improve from 4.69455
Epoch 247/1500
143/143 [==============================] - 484s 3s/step - loss: 4.9483 - root_mean_squared_error: 6.2214 - mean_absolute_error: 4.9483 - val_loss: 5.1563 - val_root_mean_squared_error: 6.7706 - val_mean_absolute_error: 5.1563

Epoch 00247: val_loss did not improve from 4.69455
Epoch 248/1500
143/143 [==============================] - 319s 2s/st


Epoch 00272: val_loss did not improve from 4.69455
Epoch 273/1500
143/143 [==============================] - 289s 2s/step - loss: 4.9148 - root_mean_squared_error: 6.3654 - mean_absolute_error: 4.9148 - val_loss: 5.6706 - val_root_mean_squared_error: 7.2174 - val_mean_absolute_error: 5.6706

Epoch 00273: val_loss did not improve from 4.69455
Epoch 274/1500
143/143 [==============================] - 276s 2s/step - loss: 4.6158 - root_mean_squared_error: 6.1187 - mean_absolute_error: 4.6158 - val_loss: 5.6966 - val_root_mean_squared_error: 7.5244 - val_mean_absolute_error: 5.6966

Epoch 00274: val_loss did not improve from 4.69455
Epoch 275/1500
143/143 [==============================] - 298s 2s/step - loss: 5.0629 - root_mean_squared_error: 6.5235 - mean_absolute_error: 5.0629 - val_loss: 6.1959 - val_root_mean_squared_error: 7.6484 - val_mean_absolute_error: 6.1959

Epoch 00275: val_loss did not improve from 4.69455
Epoch 276/1500
143/143 [==============================] - 287s 2s/ste


Epoch 00300: val_loss did not improve from 4.69455
Epoch 301/1500
143/143 [==============================] - 321s 2s/step - loss: 4.8422 - root_mean_squared_error: 6.2162 - mean_absolute_error: 4.8422 - val_loss: 5.7874 - val_root_mean_squared_error: 7.3298 - val_mean_absolute_error: 5.7874

Epoch 00301: val_loss did not improve from 4.69455
Epoch 302/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9055 - root_mean_squared_error: 6.4589 - mean_absolute_error: 4.9055 - val_loss: 5.7145 - val_root_mean_squared_error: 7.0399 - val_mean_absolute_error: 5.7145

Epoch 00302: val_loss did not improve from 4.69455
Epoch 303/1500
143/143 [==============================] - 322s 2s/step - loss: 4.8142 - root_mean_squared_error: 6.1713 - mean_absolute_error: 4.8142 - val_loss: 5.3505 - val_root_mean_squared_error: 6.8226 - val_mean_absolute_error: 5.3505

Epoch 00303: val_loss did not improve from 4.69455
Epoch 304/1500
143/143 [==============================] - 300s 2s/ste

# Compare models

### Helper functions for comparison

In [92]:
# 'Inception_regressor_01' (n_average = 40, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 17s 474ms/step - loss: 6.1535 - root_mean_squared_error: 7.8651 - mean_absolute_error: 6.1535


In [93]:
# 'Inception_regressor_02' (n_average = 1, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_02.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 466ms/step - loss: 14.0345 - root_mean_squared_error: 17.0194 - mean_absolute_error: 14.0345


In [94]:
# 'Inception_regressor_03' (n_average = 40, gaussian_noise = 0.01, MSE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_03.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 445ms/step - loss: 58.4772 - root_mean_squared_error: 7.6470 - mean_absolute_error: 5.9643


In [95]:
# 'Inception_regressor_04' (n_average = 1, gaussian_noise = 0.01, MSE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_04.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 433ms/step - loss: 221.2316 - root_mean_squared_error: 14.8739 - mean_absolute_error: 12.0842


In [96]:
# 'Inception_regressor_05' (n_average = 100, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_05.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 448ms/step - loss: 6.9294 - root_mean_squared_error: 8.5678 - mean_absolute_error: 6.9294


In [58]:
# Fully_connected_regressor_02: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'Fully_connected_regressor_02.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 12s 342ms/step - loss: 71.6239 - root_mean_squared_error: 8.4631 - mean_absolute_error: 6.5200


In [98]:
# CNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'CNN_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 420ms/step - loss: 62.4980 - root_mean_squared_error: 7.9056 - mean_absolute_error: 6.1974


In [99]:
# ResNet_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'ResNet_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 17s 473ms/step - loss: 77.0855 - root_mean_squared_error: 8.7798 - mean_absolute_error: 6.6343


In [100]:
# Encoder_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01 (LR = 0.0001, no reduction)
model_path = os.path.join(PATH_MODELS, 'Encoder_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 434ms/step - loss: 73.6694 - root_mean_squared_error: 8.5831 - mean_absolute_error: 6.6446


In [101]:
# TimeCNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'TimeCNN_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 473ms/step - loss: 62.2474 - root_mean_squared_error: 7.8897 - mean_absolute_error: 6.1511
